In [47]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()  # Optional argument, if not specified will search path.

driver.get('https://www.redbus.in/');

#time.sleep(5) # Let the user actually see something!
driver.maximize_window()
driver.execute_script("window.scrollBy(0, window.innerHeight);")

view_all = driver.find_element(By.CSS_SELECTOR, 'a[href="https://www.redbus.in/online-booking/rtc-directory"]')
view_all.click();

#driver.quit()

In [3]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
kerala = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/ksrtc-kerala"]')
kerala.click();
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

kerala_route_names_prices = []
rajasthan_route_names = []
rajasthan_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route_details")
    for route_name in route_names:
        kerala_route_names_prices.append(route_name.text)

    for kerala_route_name_price in kerala_route_names_prices:
        rajasthan_route_names.append(kerala_route_name_price.split('\n')[0])

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        rajasthan_route_links.append(href)

# Initial scrape from the first tab
scrape_data()




x_path = ''
page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(1, len(page_tabs)):  # Start from the second tab
    # Re-find the page tabs to avoid stale element reference
    
    x_path = f'/html/body/div[1]/div/div[4]/div[12]/div[{i+1}]'
    element = driver.find_element(By.XPATH, x_path)
    driver.execute_script("arguments[0].click();", element)
    time.sleep(2)  # Adjust sleep time if necessary
    
    scrape_data()

print(rajasthan_route_names)
print(rajasthan_route_links)

data = {}
count = 0
for link in rajasthan_route_links:
    driver.get(link)
    driver.maximize_window()
    
    for t in range(12):
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(1)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"lh-18")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text)

    for fare in fares:
        price.append(fare.text)

    for seat_available in seats_available:
        availability.append(seat_available.text)
    
    for i in range(len(bus_name)):
        route_name.append(rajasthan_route_names[count])
        route_link.append(link)

    count += 1

    # data = {
    #     'Route_Name':route_name,
    #     'Route_Address':route_link,
    #     'Bus_Names':bus_name,
    #     'Bus_Type':bus_type,
    #     'Time_of_Departure':departing,
    #     'Travel_Duration':duration,
    #     'Time_of_Arrival':reaching,
    #     'Star_Ratings':rating,
    #     'Fare':price,
    #     'Seats_availability':availability
    # }
    # pd.DataFrame(data,index=[0])

    print(route_name)
    print(route_link)
    print(bus_name)
    print(bus_type)
    print(departing)
    print(duration)
    print(reaching)
    print(rating)
    print(price)
    print(availability)

    con = mysql.connector.connect(
    host="localhost",
    user="root",
    password="password"
    )
    cursor = con.cursor()

    query = "create database if not exists REDBUS_DATA"
    cursor.execute(query)

    query = "use REDBUS_DATA"
    cursor.execute(query)

    query = """create table if not exists KERALA_KSRTC(route_name varchar(50), Route_Address varchar(150), Bus_Names varchar(40), bus_type varchar(50), Time_of_Departure varchar(10),
        Travel_Duration varchar(10), Time_of_Arrival varchar(10), Star_Ratings float(2,1), Fare varchar(10), Seats_availability varchar(50))"""
    cursor.execute(query)
    
    query = "insert into KERALA_KSRTC values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    for i in range(len(bus_name)+1):
        data = (route_name[i],route_link[i],bus_name[i],bus_type[i],departing[i],duration[i],reaching[i],rating[i],price[i],availability[i])
        cursor.execute(query, data)
        con.commit()






['Bangalore to Kozhikode', 'Kozhikode to Ernakulam', 'Kozhikode to Bangalore', 'Ernakulam to Kozhikode', 'Kozhikode to Mysore', 'Kozhikode to Thiruvananthapuram', 'Bangalore to Kalpetta (kerala)', 'Mysore to Kozhikode', 'Kalpetta (kerala) to Bangalore', 'Kozhikode to Thrissur', 'Bangalore to Kozhikode', 'Kozhikode to Ernakulam', 'Kozhikode to Bangalore', 'Ernakulam to Kozhikode', 'Kozhikode to Mysore', 'Kozhikode to Thiruvananthapuram', 'Bangalore to Kalpetta (kerala)', 'Mysore to Kozhikode', 'Kalpetta (kerala) to Bangalore', 'Kozhikode to Thrissur', 'Thiruvananthapuram to Kozhikode', 'Bangalore to Kannur', 'Kozhikode to Kottayam', 'Kannur to Bangalore', 'Kottayam to Kozhikode', 'Thrissur to Kozhikode', 'Kozhikode to Kalpetta (kerala)', 'Coimbatore to Ooty', 'Kalpetta (kerala) to Kozhikode']
['https://www.redbus.in/bus-tickets/bangalore-to-kozhikode', 'https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam', 'https://www.redbus.in/bus-tickets/kozhikode-to-bangalore', 'https://www.red

IndexError: list index out of range

In [13]:
# experimental code (kerala)
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
kerala = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/ksrtc-kerala"]')
kerala.click();
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

kerala_route_names = []
kerala_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        kerala_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        kerala_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(4)
    scrape_data()

print(kerala_route_names)
print(kerala_route_links)


count = 0
for link in kerala_route_links:
    driver.get(link)
    driver.maximize_window()
    
    for t in range(12):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(1)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text)[-4:].strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(kerala_route_names[count])
        route_link.append(link)

    count += 1


    print(route_name)
    print(route_link)
    print(bus_name)
    print(bus_type)
    print(departing)
    print(duration)
    print(reaching)
    print(rating)
    print(price)
    print(availability)

    con = mysql.connector.connect(
    host="localhost",
    user="root",
    password="password"
    )
    cursor = con.cursor()

    query = "create database if not exists REDBUS_DATA"
    cursor.execute(query)

    query = "use REDBUS_DATA"
    cursor.execute(query)

    query = """create table if not exists KERALA_KSRTC(id INT AUTO_INCREMENT PRIMARY KEY, route_name varchar(70), route_link varchar(180), busname varchar(100), bustype varchar(50), departing_time DATETIME,
        duration varchar(20), reaching_time DATETIME, star_rating float(2,1), price DECIMAL(6, 2), seats_available INT)"""
    cursor.execute(query)
    
    query = """insert into KERALA_KSRTC(route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available) 
            values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    for i in range(len(bus_name)):
        data = (route_name[i],route_link[i],bus_name[i],bus_type[i],departing[i],duration[i],reaching[i],rating[i],price[i],availability[i])
        cursor.execute(query, data)
        con.commit()
    
    

['Bangalore to Kozhikode', 'Kozhikode to Ernakulam', 'Kozhikode to Bangalore', 'Ernakulam to Kozhikode', 'Kozhikode to Mysore', 'Kozhikode to Thiruvananthapuram', 'Bangalore to Kalpetta (kerala)', 'Mysore to Kozhikode', 'Kalpetta (kerala) to Bangalore', 'Kozhikode to Thrissur', 'Thiruvananthapuram to Kozhikode', 'Bangalore to Kannur', 'Kozhikode to Kottayam', 'Kannur to Bangalore', 'Kottayam to Kozhikode', 'Thrissur to Kozhikode', 'Kozhikode to Kalpetta (kerala)', 'Coimbatore to Ooty', 'Kalpetta (kerala) to Kozhikode']
['https://www.redbus.in/bus-tickets/bangalore-to-kozhikode', 'https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam', 'https://www.redbus.in/bus-tickets/kozhikode-to-bangalore', 'https://www.redbus.in/bus-tickets/ernakulam-to-kozhikode', 'https://www.redbus.in/bus-tickets/kozhikode-to-mysore', 'https://www.redbus.in/bus-tickets/kozhikode-to-thiruvananthapuram', 'https://www.redbus.in/bus-tickets/bangalore-to-kalpetta', 'https://www.redbus.in/bus-tickets/mysore-to-kozh

In [15]:
# experimental code (kadamba ktcl)
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
telangana = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/ktcl"]')
telangana.click();
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

kadamba_route_names = []
kadamba_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        kadamba_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        kadamba_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(2)
    scrape_data()

print(kadamba_route_names)
print(kadamba_route_links)


count = 0
for link in kadamba_route_links:
    driver.get(link)
    driver.maximize_window()
    

    for t in range(16):     #scrolling 16 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(1)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text)[-4:].strip())

    for seat_available in seats_available:
        availability.append(seat_available.text)
    
    for i in range(len(bus_name)):
        route_name.append(kadamba_route_names[count])
        route_link.append(link)

    count += 1


    print(route_name)
    print(route_link)
    print(bus_name)
    print(bus_type)
    print(departing)
    print(duration)
    print(reaching)
    print(rating)
    print(price)
    print(availability)

    # con = mysql.connector.connect(
    # host="localhost",
    # user="root",
    # password="password"
    # )
    # cursor = con.cursor()

    # query = "create database if not exists REDBUS_DATA"
    # cursor.execute(query)

    # query = "use REDBUS_DATA"
    # cursor.execute(query)

    # query = """create table if not exists KADAMBA_KTCL(route_name varchar(70), Route_Address varchar(180), Bus_Names varchar(100), bus_type varchar(50), Time_of_Departure varchar(15),
    #     Travel_Duration varchar(20), Time_of_Arrival varchar(15), Star_Ratings varchar(10), Fare_in_INR varchar(15), Seats_availability varchar(50))"""
    # cursor.execute(query)
    
    # query = "insert into KADAMBA_KTCL values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    # for i in range(len(bus_name)):
    #     data = (route_name[i],route_link[i],bus_name[i],bus_type[i],departing[i],duration[i],reaching[i],rating[i],price[i],availability[i])
    #     cursor.execute(query, data)
    #     con.commit()
    
    

['Pune to Goa', 'Goa to Pune', 'Mumbai to Goa', 'Goa to Mumbai', 'Pandharpur to Goa', 'Bangalore to Goa', 'Goa to Pandharpur', 'Belagavi to Goa', 'Goa to Bangalore', 'Solapur to Goa', 'Goa to Kolhapur(Maharashtra)', 'Goa to Solapur', 'Goa to Sangola (Solapur)', 'Sangola (Solapur) to Goa', 'Calangute (goa) to Goa Airport', 'Goa to Sangli', 'Calangute (goa) to Mopa Airport', 'Mopa Airport to Calangute (goa)', 'Ponda to Belagavi', 'Goa to Miraj', 'Goa Airport to Calangute (goa)', 'Marcel to Belagavi', 'Shivamogga to Goa', 'Goa to Mopa Airport', 'Goa to Satara', 'Belagavi to Marcel', 'Mopa Airport to Goa', 'Shirdi to Goa', 'Goa to Shivamogga', 'Goa to Shirdi', 'Goa to Goa Airport', 'Margao to Mopa Airport', 'Goa Airport to Goa', 'Mopa Airport to Margao', 'Belagavi to Saquelim', 'Panaji to Mopa Airport', 'Saquelim to Belagavi', 'Calangute (goa) to Goa', 'Calangute (goa) to Panaji', 'Goa Airport to Panaji']
['https://www.redbus.in/bus-tickets/pune-to-goa', 'https://www.redbus.in/bus-tickets/

In [2]:
# experimental code (Rajasthan)
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
rajasthan = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/rsrtc"]')
rajasthan.click();
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

rajasthan_route_names = []
rajasthan_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        rajasthan_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        rajasthan_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(4)
    scrape_data()

print(rajasthan_route_names)
print(rajasthan_route_links)


count = 0
for link in rajasthan_route_links:
    driver.get(link)
    driver.maximize_window()
    
    for t in range(17):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(1)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text)
    
    for i in range(len(bus_name)):
        route_name.append(rajasthan_route_names[count])
        route_link.append(link)

    count += 1


    print(route_name)
    print(route_link)
    print(bus_name)
    print(bus_type)
    print(departing)
    print(duration)
    print(reaching)
    print(rating)
    print(price)
    print(availability)

    # con = mysql.connector.connect(
    # host="localhost",
    # user="root",
    # password="password"
    # )
    # cursor = con.cursor()

    # query = "create database if not exists REDBUS_DATA"
    # cursor.execute(query)

    # query = "use REDBUS_DATA"
    # cursor.execute(query)

    # query = """create table if not exists RAJASTHAN_RSRTC(route_name varchar(70), Route_Address varchar(180), Bus_Names varchar(100), bus_type varchar(50), Time_of_Departure varchar(15),
    #     Travel_Duration varchar(20), Time_of_Arrival varchar(15), Star_Ratings varchar(10), Fare_in_INR varchar(15), Seats_availability varchar(50))"""
    # cursor.execute(query)
    
    # query = "insert into RAJASTHAN_RSRTC values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    # for i in range(len(bus_name)):
    #     data = (route_name[i],route_link[i],bus_name[i],bus_type[i],departing[i],duration[i],reaching[i],rating[i],price[i],availability[i])
    #     cursor.execute(query, data)
    #     con.commit()
    
    

['Jodhpur to Ajmer', 'Beawar (Rajasthan) to Jaipur (Rajasthan)', 'Udaipur to Jodhpur', 'Jaipur (Rajasthan) to Jodhpur', 'Sikar to Jaipur (Rajasthan)', 'Kishangarh to Jaipur (Rajasthan)', 'Aligarh (uttar pradesh) to Jaipur (Rajasthan)', 'Jodhpur to Beawar (Rajasthan)', 'Kota(Rajasthan) to Jaipur (Rajasthan)', 'Jaipur (Rajasthan) to Aligarh (uttar pradesh)', 'Jaipur (Rajasthan) to Kota(Rajasthan)', 'Pali (Rajasthan) to Udaipur', 'Udaipur to Pali (Rajasthan)', 'Sikar to Bikaner', 'Jaipur (Rajasthan) to Bharatpur', 'Kishangarh to Jodhpur', 'Jaipur (Rajasthan) to Bhilwara', 'Kota(Rajasthan) to Udaipur', 'Jaipur (Rajasthan) to Pilani', 'Jaipur (Rajasthan) to Mathura', 'Bikaner to Sikar']
['https://www.redbus.in/bus-tickets/jodhpur-to-ajmer', 'https://www.redbus.in/bus-tickets/beawer-to-jaipur', 'https://www.redbus.in/bus-tickets/udaipur-to-jodhpur', 'https://www.redbus.in/bus-tickets/jaipur-to-jodhpur', 'https://www.redbus.in/bus-tickets/sikar-to-jaipur', 'https://www.redbus.in/bus-tickets/k

In [10]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
kerala = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/ksrtc-kerala"]')
kerala.click();
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")


rajasthan_route_names = []
rajasthan_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        rajasthan_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        rajasthan_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(4)
    scrape_data()

print(rajasthan_route_names)
print(rajasthan_route_links)

['Bangalore to Kozhikode', 'Kozhikode to Ernakulam', 'Kozhikode to Bangalore', 'Ernakulam to Kozhikode', 'Kozhikode to Mysore', 'Kozhikode to Thiruvananthapuram', 'Bangalore to Kalpetta (kerala)', 'Mysore to Kozhikode', 'Kalpetta (kerala) to Bangalore', 'Kozhikode to Thrissur', 'Thiruvananthapuram to Kozhikode', 'Bangalore to Kannur', 'Kozhikode to Kottayam', 'Kannur to Bangalore', 'Kottayam to Kozhikode', 'Thrissur to Kozhikode', 'Kozhikode to Kalpetta (kerala)', 'Coimbatore to Ooty', 'Kalpetta (kerala) to Kozhikode']
['https://www.redbus.in/bus-tickets/bangalore-to-kozhikode', 'https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam', 'https://www.redbus.in/bus-tickets/kozhikode-to-bangalore', 'https://www.redbus.in/bus-tickets/ernakulam-to-kozhikode', 'https://www.redbus.in/bus-tickets/kozhikode-to-mysore', 'https://www.redbus.in/bus-tickets/kozhikode-to-thiruvananthapuram', 'https://www.redbus.in/bus-tickets/bangalore-to-kalpetta', 'https://www.redbus.in/bus-tickets/mysore-to-kozh

In [4]:
import mysql.connector
con = mysql.connector.connect(
    host="localhost",
    user="root",
    password="password"
)
cursor = con.cursor()

query = "create database if not exists REDBUS_DATA"
cursor.execute(query)

query = "use REDBUS_DATA"
cursor.execute(query)

query = """create table if not exists KERALA_KSRTC(route_name varchar(50), Route_Address varchar(150), Bus_Names varchar(40), bus_type varchar(50), Time_of_Departure varchar(10),
        Travel_Duration varchar(10), Time_of_Arrival varchar(10), Star_Ratings float(4), Fare varchar(10), Seats_availability varchar(50))"""
cursor.execute(query)



In [56]:
for link in rajasthan_route_links:
    print(link)
    driver.get(link)
    driver.maximize_window()
    time.sleep(3)
    
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"lh-18")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text)

    for fare in fares:
        price.append(fare.text)

    for seat_available in seats_available:
        availability.append(seat_available.text)
    break
print(bus_name)
print(bus_type)
print(departing)
print(duration)
print(reaching)
print(rating)
print(price)
print(availability)



https://www.redbus.in/bus-tickets/bangalore-to-kozhikode


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF6845DEEA2+31554]
	(No symbol) [0x00007FF684557ED9]
	(No symbol) [0x00007FF68441872A]
	(No symbol) [0x00007FF6843FFA9C]
	(No symbol) [0x00007FF6843FF960]
	(No symbol) [0x00007FF68441AD51]
	(No symbol) [0x00007FF6844AC079]
	(No symbol) [0x00007FF68448CDD3]
	(No symbol) [0x00007FF68445A33B]
	(No symbol) [0x00007FF68445AED1]
	GetHandleVerifier [0x00007FF6848E8B1D+3217341]
	GetHandleVerifier [0x00007FF684935AE3+3532675]
	GetHandleVerifier [0x00007FF68492B0E0+3489152]
	GetHandleVerifier [0x00007FF68468E776+750614]
	(No symbol) [0x00007FF68456375F]
	(No symbol) [0x00007FF68455EB14]
	(No symbol) [0x00007FF68455ECA2]
	(No symbol) [0x00007FF68454E16F]
	BaseThreadInitThunk [0x00007FFCCF22257D+29]
	RtlUserThreadStart [0x00007FFCCFE4AF28+40]


In [9]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()  # Optional argument, if not specified will search path.

driver.get('https://www.redbus.in/');

#time.sleep(5) # Let the user actually see something!
driver.maximize_window()
driver.execute_script("window.scrollBy(0, window.innerHeight);")

view_all = driver.find_element(By.CSS_SELECTOR, 'a[href="https://www.redbus.in/online-booking/rtc-directory"]')
view_all.click();
driver.maximize_window()
#driver.quit()

In [18]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
kerala = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/ksrtc-kerala"]')
kerala.click();
time.sleep(4)
driver.execute_script("window.scrollBy(0, window.innerHeight);")

kerala_route_names_prices = []
rajasthan_route_names = []
rajasthan_route_links = []
count = 1

page_tabs = driver.find_elements(By.CLASS_NAME,"DC_117_pageTabs ")
#page_tabs_nums = len(page_tabs)
#print(page_tabs_nums)

#for page_tab in page_tabs:
#    print(page_tab.text)


for i in range(len(page_tabs)):
    # Re-find the page tabs to avoid stale element reference
    page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")

    if count > 1:
        page_tabs[i].click()

    #for i in range(page_tabs_nums):
    #    driver.find_elements(By.CLASS_NAME,"DC_117_pageTabs ").text

    route_names = driver.find_elements(By.CLASS_NAME,"route_details")

    for route_name in route_names:
        kerala_route_names_prices.append(route_name.text)
        #print(route_name.text)

    for kerala_route_name_price in kerala_route_names_prices:
        rajasthan_route_names.append(kerala_route_name_price.split('\n')[0])

    route_links = driver.find_elements(By.CLASS_NAME,"route")

    hrefs = [route_link.get_attribute('href') for route_link in route_links]

    for href in hrefs:
        rajasthan_route_links.append(href)

    count += 1

print(rajasthan_route_names)
print(rajasthan_route_links)



ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (694, 1470)
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF6845DEEA2+31554]
	(No symbol) [0x00007FF684557ED9]
	(No symbol) [0x00007FF68441872A]
	(No symbol) [0x00007FF68447012E]
	(No symbol) [0x00007FF68446DAF2]
	(No symbol) [0x00007FF68446AF8B]
	(No symbol) [0x00007FF68446A156]
	(No symbol) [0x00007FF68445C151]
	(No symbol) [0x00007FF68448D02A]
	(No symbol) [0x00007FF68445BA76]
	(No symbol) [0x00007FF68448D240]
	(No symbol) [0x00007FF6844AC977]
	(No symbol) [0x00007FF68448CDD3]
	(No symbol) [0x00007FF68445A33B]
	(No symbol) [0x00007FF68445AED1]
	GetHandleVerifier [0x00007FF6848E8B1D+3217341]
	GetHandleVerifier [0x00007FF684935AE3+3532675]
	GetHandleVerifier [0x00007FF68492B0E0+3489152]
	GetHandleVerifier [0x00007FF68468E776+750614]
	(No symbol) [0x00007FF68456375F]
	(No symbol) [0x00007FF68455EB14]
	(No symbol) [0x00007FF68455ECA2]
	(No symbol) [0x0

In [23]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
kerala = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/ksrtc-kerala"]')
kerala.click();
time.sleep(4)
driver.execute_script("window.scrollBy(0, window.innerHeight);")

kerala_route_names_prices = []
rajasthan_route_names = []
rajasthan_route_links = []


page_tabs = driver.find_elements(By.CLASS_NAME,"DC_117_pageTabs ")
#page_tabs_nums = len(page_tabs)
#print(page_tabs_nums)

#for page_tab in page_tabs:
#    print(page_tab.text)


for i in range(len(page_tabs)):


    #for i in range(page_tabs_nums):
    #    driver.find_elements(By.CLASS_NAME,"DC_117_pageTabs ").text

    route_names = driver.find_elements(By.CLASS_NAME,"route_details")

    for route_name in route_names:
        kerala_route_names_prices.append(route_name.text)
        #print(route_name.text)

    for kerala_route_name_price in kerala_route_names_prices:
        rajasthan_route_names.append(kerala_route_name_price.split('\n')[0])

    route_links = driver.find_elements(By.CLASS_NAME,"route")

    hrefs = [route_link.get_attribute('href') for route_link in route_links]

    for href in hrefs:
        rajasthan_route_links.append(href)

    print(rajasthan_route_names)
    print(rajasthan_route_links)

        # Re-find the page tabs to avoid stale element reference
    page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
    #print(page_tabs)

    if i < (len(page_tabs)-1):
        page_tabs[i+1].click()
    
    

print(rajasthan_route_names)
print(rajasthan_route_links)



['Bangalore to Kozhikode', 'Kozhikode to Ernakulam', 'Kozhikode to Bangalore', 'Ernakulam to Kozhikode', 'Kozhikode to Mysore', 'Kozhikode to Thiruvananthapuram', 'Bangalore to Kalpetta (kerala)', 'Mysore to Kozhikode', 'Kalpetta (kerala) to Bangalore', 'Kozhikode to Thrissur']
['https://www.redbus.in/bus-tickets/bangalore-to-kozhikode', 'https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam', 'https://www.redbus.in/bus-tickets/kozhikode-to-bangalore', 'https://www.redbus.in/bus-tickets/ernakulam-to-kozhikode', 'https://www.redbus.in/bus-tickets/kozhikode-to-mysore', 'https://www.redbus.in/bus-tickets/kozhikode-to-thiruvananthapuram', 'https://www.redbus.in/bus-tickets/bangalore-to-kalpetta', 'https://www.redbus.in/bus-tickets/mysore-to-kozhikode', 'https://www.redbus.in/bus-tickets/kalpetta-to-bangalore', 'https://www.redbus.in/bus-tickets/kozhikode-to-thrissur']


ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (694, 664)
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF6845DEEA2+31554]
	(No symbol) [0x00007FF684557ED9]
	(No symbol) [0x00007FF68441872A]
	(No symbol) [0x00007FF68447012E]
	(No symbol) [0x00007FF68446DAF2]
	(No symbol) [0x00007FF68446AF8B]
	(No symbol) [0x00007FF68446A156]
	(No symbol) [0x00007FF68445C151]
	(No symbol) [0x00007FF68448D02A]
	(No symbol) [0x00007FF68445BA76]
	(No symbol) [0x00007FF68448D240]
	(No symbol) [0x00007FF6844AC977]
	(No symbol) [0x00007FF68448CDD3]
	(No symbol) [0x00007FF68445A33B]
	(No symbol) [0x00007FF68445AED1]
	GetHandleVerifier [0x00007FF6848E8B1D+3217341]
	GetHandleVerifier [0x00007FF684935AE3+3532675]
	GetHandleVerifier [0x00007FF68492B0E0+3489152]
	GetHandleVerifier [0x00007FF68468E776+750614]
	(No symbol) [0x00007FF68456375F]
	(No symbol) [0x00007FF68455EB14]
	(No symbol) [0x00007FF68455ECA2]
	(No symbol) [0x00